In [1]:
import os
import streamlit as st
from dotenv import load_dotenv
from PyPDF2 import PdfReader
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.chat_models import AzureChatOpenAI as AzureOpenAI
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import UnstructuredURLLoader
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.chains import RetrievalQA
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.agents import Tool, initialize_agent

In [2]:
load_dotenv()
 # Define OpenAI credentials
openai_api_type = os.getenv("OPENAI_API_TYPE")
openai_api_version = os.getenv("OPENAI_API_VERSION")
openai_api_base = os.getenv("OPENAI_API_BASE")
openai_api_key = os.getenv("OPENAI_API_KEY")
model_deployment_id = os.getenv("MODEL_DEPLOYMENT_ID")
# Set environment variables
if openai_api_base is not None:
    os.environ["OPENAI_API_BASE"] = openai_api_base
if openai_api_key is not None:
    os.environ["OPENAI_API_KEY"] = openai_api_key
if openai_api_type is not None:
    os.environ["OPENAI_API_TYPE"] = openai_api_type
if openai_api_version is not None:
     os.environ["OPENAI_API_VERSION"] = str(openai_api_version)

In [3]:
embeddings = OpenAIEmbeddings(
client=any,
deployment='embeddings',
model='text-embedding-ada-002',
openai_api_type="azure",
openai_api_base="https://copilot-openai.openai.azure.com",
openai_api_version="2023-05-15",
openai_api_key=os.environ["OPENAI_API_KEY"],
chunk_size=1
)

In [27]:
uploaded_docs = FAISS.load_local("faiss_uploaded_docs", embeddings)
llm1 = AzureOpenAI(deployment_name='trouble-buddy', model_name='gpt-3.5-turbo', temperature=0, client=any)
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

qa1 = RetrievalQA.from_chain_type(
    llm=llm1,
    retriever=uploaded_docs.as_retriever(),
    memory=memory
)

In [28]:
kb_sre = FAISS.load_local("livro_google", embeddings)
#llm = AzureOpenAI(model_kwargs={'engine':'trouble-buddy'}, client=any)
llm = AzureOpenAI(deployment_name='trouble-buddy', model_name='gpt-3.5-turbo', temperature=0.0, client=any)
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

qa = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=kb_sre.as_retriever(),
    chain_type='stuff',
    memory=memory
)


In [29]:
uploaded_docs.similarity_search("What happened During Sprint 222 on Azure that caused a service disruption?")

[Document(page_content='Azur e DevOps Outage in South Brazil\n\ueddbEMEric Mattingly ,\n6/2/2023, 3:59 PM\nOn W ednesday, 24 May 2023,\nAzure DevOps had an outage\nof one of our South Brazil\n(SBR) scale units which lasted\nfrom 12:10 to 22:31 UT C\n(~10.5 hours). W e understand\nhow impactful Azure DevOps\noutages can be, and sincerely\napologize to all the impacted\ncustomers.\nBack ground\nAzure DevOps engineers\nsometimes need to take a\npoint-in-time snapshot of a\nproduction database to\ninvestigate customer\nescalations or evaluate\npotential performance\nimprovements. T o ensure\nthese snapshot databases get\ncleaned up, a background job\nruns every day and deletes\nthem once they pass a certain\nage.\nWhat trigger ed the\noutage?\nDuring Sprint 222, we\nupgraded our code base to\nreplace the deprecated\nMicrosoft.Azure.Managment.*\npackages with the supported\nAzure.R esourceManager.*\nNuGet packages. This\nresulted in a large pullEvent information\nState\nResolvedStart\n5/24/

In [30]:
qa1.run("What happened?")

"There was an outage of one of Azure DevOps' South Brazil scale units which lasted for approximately 10.5 hours. The outage was caused by a background job that runs every day to delete point-in-time snapshot databases once they pass a certain age. During Sprint 222, Azure DevOps engineers upgraded their code base which resulted in a large pull of databases for the scale unit. Starting at that moment, the scale unit was unable to process any customer traffic. The issue was quickly understood, and the engineers got to work restoring the databases. There was no data loss experienced, but it took over 10 hours to recover. During this time, the web server's health probe was failing and was thus not receiving customer traffic from the load balancer. Once all databases were healthy, users were gradually unblocked to ramp the system back up."

In [31]:
qa1.run("for how long did the incident last?")

'The incident lasted for approximately 10.5 hours, from 12:10 to 22:31 UTC on Wednesday, May 24, 2023.'

In [8]:
tools = [
    Tool(
        name = "Upload de documentos",
        func=qa1.run,
        description=(
            'Use this tool when answering questions about the uploaded documents.'
        )
    ),
    Tool(
        name="Base de Conhecimento sobre o livro Building Secure and Reliable Systems do Google",
        func=qa.run,
        description=(
            'Use this tool when answering questions about SRE (Site Reliability Engineeing) and about best practices for building secure and reliable systems.'
        )
    )
]
agente = initialize_agent(
    agent='chat-conversational-react-description',
    tools=tools,
    llm=llm,
    verbose=True,
    max_interactions=3,
    early_stopping_method='generate',
    memory=memory,
    handle_parsing_errors=True
)

In [33]:
agente("What are the SRE principles? associated with Observability?")



> Entering new AgentExecutor chain...
{
    "action": "Base de Conhecimento sobre o livro Building Secure and Reliable Systems do Google",
    "action_input": "What are the SRE principles associated with Observability?"
}
Observation: The given context does not provide a direct answer to the user's question. However, the context mentions the importance of recording observations and expectations during an investigation, which can help in understanding the steps taken during the investigation and can be useful for other team members to participate in or take over the investigation. This is related to the principle of observability, which is one of the key principles of SRE. Observability refers to the ability to understand the internal state of a system based on its external outputs. By recording observations and expectations, SREs can improve the observability of the system and make it easier to debug and troubleshoot issues. Additionally, the context mentions the importance of monito

{'input': 'What are the SRE principles? associated with Observability?',
 'chat_history': [HumanMessage(content='What is the uploaded document all about?', additional_kwargs={}, example=False),
  AIMessage(content="I apologize for the confusion earlier. Unfortunately, I don't have any information about an uploaded document since I don't have access to it. However, if you can provide me with more context or information about what you are looking for, I will do my best to assist you.", additional_kwargs={}, example=False),
  HumanMessage(content='What incident happened on Azure that is decribed in the document uploaded?', additional_kwargs={}, example=False),
  AIMessage(content="You're welcome! If you have any questions in the future, don't hesitate to ask. Have a great day!", additional_kwargs={}, example=False),
  HumanMessage(content='What happened During Sprint 222 on Azure that caused a service disruption?', additional_kwargs={}, example=False),
  AIMessage(content='During Sprint 2